In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from exod.utils.path import data_results
import pandas as pd
from glob import glob
import re
from itertools import combinations

from astropy.visualization import hist
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.stats import skew, kurtosis
from tqdm import tqdm

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 20)

In [ ]:
filepath = '../data/results_combined/t_10s_7k_obs/'
regex = re.compile(r'\d{10}')

In [ ]:
# Detected Regions
dfs = []
for f in glob(f'{filepath}*/*/*regions.csv'):
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    df['obsid'] = obsid
    df['subset'] = int(f.split('/')[-2][-1])
    #df = df.T  
    #df['obsid'] = obsid
    dfs.append(df)

df_regions = pd.concat(dfs, ignore_index=True)
df_regions


In [ ]:
tab_xmm = Table.read('../data/util/4XMM_slim_DR13cat_v1.0.fits')
skycoord_xmm = SkyCoord(ra=tab_xmm['SC_RA'], dec=tab_xmm['SC_DEC'], unit=u.deg)
sky_coords = SkyCoord(ra=df_regions['ra_deg'].values, dec=df_regions['dec_deg'].values, unit='deg', frame='icrs')

cmatch = sky_coords.match_to_catalog_sky(skycoord_xmm)
tab_cmatch = Table(cmatch)
tab_cmatch.rename_columns(names=tab_cmatch.colnames, new_names=['idx', 'sep2d', 'dist3d'])
tab_cmatch['sep2d_arcsec'] = tab_cmatch['sep2d'].to(u.arcsec)
tab_cmatch['idx_orig'] = np.arange(len(tab_cmatch))

tab_xmm_cmatch = tab_xmm[tab_cmatch['idx']]
tab_xmm_cmatch['SEP'] = tab_cmatch['sep2d']
tab_xmm_cmatch['SEP_ARCSEC'] = tab_xmm_cmatch['SEP'] * 3600
tab_xmm_cmatch

In [ ]:
for i, row in df_regions.iterrows():
    obsid   = row['obsid']
    subset  = row['subset']
    label   = row['label']
    ra      = row['ra']
    dec     = row['dec']
    ra_deg  = row['ra_deg']
    dec_deg = row['dec_deg']

    csv_lc  = glob(f'{filepath}{obsid}/subset_{subset}/lcs.csv')
    csv_evt = glob(f'{filepath}{obsid}/subset_{subset}/evt_info.csv')
    img_png = glob(f'{filepath}{obsid}/subset_{subset}/*detection_img.png')

    if len(img_png) > 1:
        print('MULTIPLE SUBSETS!!!')
    
    df_lc = pd.read_csv(csv_lc[0])
    df_evt = pd.read_csv(csv_evt[0])
    
    
    date  = df_evt.iloc[4].iloc[1]
    obj   = df_evt.iloc[5].iloc[1]

    tab_closest = tab_xmm_cmatch[i]
    c_ra      = tab_closest['SC_RA']
    c_dec     = tab_closest['SC_DEC']
    c_webpage = tab_closest['WEBPAGE_URL']    
    c_var     = tab_closest['SC_VAR_FLAG']
    c_sep     = tab_closest['SEP_ARCSEC']
    c_flux_8  = tab_closest['SC_EP_8_FLUX']


    
    n = label - 1
    if np.max(df_lc[f'n_{n}']) > 10:
            
        esasky = f'https://sky.esa.int/esasky/?target={ra_deg}-{dec_deg}&hips=XMM-Newton+EPIC+color&fov=0.25&cooframe=J2000&sci=true&lang=en'
        simbad = f'https://simbad.cds.unistra.fr/simbad/sim-basic?Ident={ra_deg}+{dec_deg}'
        xsa    = f'https://nxsa.esac.esa.int/nxsa-web/#obsid={obsid}'
        print(f'{i}/{len(df_regions)} | {obsid} | {obj} | {date} | {ra} {dec} {ra_deg} {dec_deg}')
        print(f'Closest DR13 ({c_sep:.2f}"): {c_ra} {c_dec} | SC_VAR_FLAG={c_var} | SC_EP_8_FLUX (0.2-12.0) = {c_flux_8:.3e}')
        print(f'{c_webpage}')
        print(f'{esasky}')
        print(f'{simbad}')
        print(f'{xsa}')
        
        plt.figure(figsize=(15,3))
        plt.plot(df_lc['time'], df_lc[f'n_{n}'], color='black', label='Observed (n)', lw=1.0)
        plt.plot(df_lc['time'], df_lc[f'mu_{n}'], color='red', label=r'Expected ($\mu$)', lw=1.0)
        plt.legend()
        #plt.tight_layout()
        plt.show()
    
        # im = plt.imread(img_png[0])
        #plt.figure(figsize=(15,15))
        #plt.imshow(im)
        #plt.axis('off')
        ##plt.tight_layout()
        #plt.show()

In [ ]:
# Load Lightcurves
regex = re.compile(r'\d{10}')
all_res = []
for f in glob(f'{filepath}*/*/*lcs.csv'):
    print(f)
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    plt.figure(figsize=(15,3))
    plt.plot(df['time'], df['B_peak_0'], label='B_Peak', lw=1.0)
    plt.plot(df['time'], df['B_eclipse_0'], label=r'B_eclipse', lw=1.0)
    plt.yscale('log')
    plt.legend()
    plt.show()

In [ ]:
# Load Lightcurves
all_res = []
for f in glob(f'{filepath}*/*/*lcs.csv'):
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    print(f'obsid={obsid}')
    plt.figure(figsize=(15,3))
    plt.plot(df['time'], df['n_0'], color='black', label='Observed (n)', lw=1.0)
    plt.plot(df['time'], df['mu_0'], color='red', label=r'Expected ($\mu$)', lw=1.0)
    plt.legend()
    plt.show()
    if df.shape[1] > 7:
        n = int((df.shape[1] - 7) / 4)
        print(f'n={n}')
        for i in range(1,n):
            plt.figure(figsize=(15,3))
            plt.plot(df['time'], df[f'n_{i}'], color='black', label='Observed (n)', lw=1.0)
            plt.plot(df['time'], df[f'mu_{i}'], color='red', label=r'Expected ($\mu$)', lw=1.0)
            plt.legend()
            plt.show()